In [ ]:
import geopandas as gpd
from shapely import Point
import matplotlib.pyplot as plt
plt.rcParams['font.family']=['monospace']

In [ ]:
url = 'https://www.ordnancesurvey.co.uk/legacy/docs/gps/CompleteTrigArchive.zip'
trigpoints = gpd.read_file(url)

for var in ['EASTING','NORTHING','HEIGHT']:
    trigpoints[var] = trigpoints[var].astype('float32')
    
trigpoints['geometry'] = trigpoints.apply(lambda q: Point(q['EASTING'],
                                                          q['NORTHING']),
                                          axis=1)
trigpoints = trigpoints.set_crs(27700)

In [ ]:
fig, ax = plt.subplots(figsize=[10,10])

trigpoints.loc[(trigpoints['DESTROYED MARK INDICATOR']=='0')
               &
               (trigpoints['TYPE OF MARK'] == 'PILLAR')].plot(ax=ax,
                                                              markersize=0.1,
                                                              column='HEIGHT',
                                                              cmap='gist_earth',
                                                              legend=True,
                                                              legend_kwds={
                                                                  'label':'Elevation (m)',
                                                                  'shrink':0.9,
                                                                  }
                                                              )
_ = ax.set_title('Trig Points of\nGreat Britain',
                 loc='left',
                 y=0.95,
                 fontsize=16,
                 )

ax.set_axis_off()

fig.savefig('day1.png', bbox_inches='tight')